In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import transforms
from torch import optim


In [2]:
# 定义卷积运算
def corr2d(X,K): # X 输入(H, W) K 卷积核 (k_h, k_w)
    H, W = X.shape
    k_h, k_w = K.shape
    # 初始化结果矩阵
    Y = torch.zeros((H-k_h+1, W-k_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+k_h, j:j+k_w]*K).sum()
    return Y
    

In [3]:
# 将卷积运算封装
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [4]:
# 卷积用来边缘检测
X = torch.ones(6, 8)
X[:, 2:6] = 0
K = torch.tensor([[1,-1]])
Y = corr2d(X,K)

In [5]:
# 反向传播
# 构造一个核数组形状是(1,2)的二位卷积层
conv2d = Conv2D(kernel_size=(1,2))
step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat-Y)**2).sum()
    l.backward()
    
    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad
    # 梯度清0
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if (i+1)%5 == 0:
        pass
        # print('Step %d, loss %.3f' % (i+1, l.item()))


In [6]:
# 定义一个函数来计算卷积层，它对输入和输出做相应的升维和降维
def comp_conv2d(conv2d, X):
    X = X.view((1,1) + X.shape)
    Y = conv2d(X)
    return Y.view(Y.shape[2:])

conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3,padding=1,stride=2)
X = torch.rand(8,8)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [7]:
# 多通道卷积
def corr2d_multi_in(X, K):
    # X [512 3 32 32]
    # K[3 3 3]
    res = corr2d(X[0,:,:], K[0, :,:])
    for i in range(1, X.shape[0]):
        res += corr2d(X[i, :, :], K[i, :, :])
    return res

In [8]:
X = torch.tensor([[[0,1,2],[3,4,5],[6,7,8]],[[1,2,3],[4,5,6],[7,8,9]]])
K = torch.tensor([[[0,1],[2,3]],[[1,2],[3,4]]])
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [9]:
def corr2d_multi_in_out(X,K):
    # X [512 3 32 32]
    # K[32 3 3 3]
    res = []
    for x in X:
        res.append(torch.stack([corr2d_multi_in(x,k) for k in K]))
    return torch.stack(res)

In [10]:
# 自定义池化
def pool2d(X, pool_size, mode='max'):
    X = X.float()
    p_h, p_w = pool_size
    Y = torch.zeros(X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i: i+p_h, j: j+p_w].max()
            elif mode == 'avg':
                Y[i,j] = X[i: i+p_h, j: j+p_w].mean()
    return Y

In [11]:
class MyConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size): 
        super(MyConv2D, self).__init__()
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)
        # weight[32 3 3 3]
        self.weight = nn.Parameter(torch.randn((out_channels, in_channels) + kernel_size))
        # bias[32 1 1]
        self.bias = nn.Parameter(torch.randn(out_channels, 1, 1))
        
    def forward(self, x):
        # x [512 3 32 32]
        return corr2d_multi_in_out(x, self.weight) + self.bias

In [12]:
# 自定义卷积层
class MyConvModule(nn.Module):
    def __init__(self):
        super(MyConvModule, self).__init__()
        self.conv = nn.Sequential(
            MyConv2D(in_channels=3, out_channels=32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        self.fc = nn.Linear(32, num_classes)
        
    def forward(self, X):
        out = self.conv(X)
        out = F.avg_pool2d(out, 30)
        out = out.squeeze()
        out = self.fc(out)
        return out

In [13]:
# 训练函数
def train_epoch(net, data_loader, device):
    net.train() # 指定当前为训练模式
    train_batch_num = len(data_loader) # 记录共有多少个batch
    total_loss = 0 # 记录loss
    correct = 0 # 记录共有多少个样本被正确分类
    sample_num = 0 # 记录样本总数
    
    # 遍历每个batch进行训练
    for batch_idx, (data, target) in enumerate(data_loader):
        data = data.to(device).float() # 将图片放入指定的device中
        target = target.to(device).long() # 将图片标签放入到指定的device中
        optimizer.zero_grad() # 将当前梯度清零
        output = net(data)
        print('output.shape', output.shape, 'target.shape', target.shape)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        prediction = torch.argmax(output, 1) # 找出每个样本值最大的idx, 即岱庙预测此图片属于哪个类别
        correct += (prediction == target).sum().item()
        sample_num += len(prediction)
    
    loss = total_loss / train_batch_num
    acc = correct / sample_num
    return loss, acc

In [14]:
# 测试函数
def test_epoch(net, data_loader, device):
    net.eval() # 指定当前模式为测试模式
    test_batch_num = len(data_loader)
    total_loss = 0
    correct = 0
    sample_num = 0
    
    # 指定不进行梯度变化
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            data = data.to(device).float()
            target = target.to(device).long()
            output = net(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            prediction = torch.argmax(output, 1)
            correct += (prediction == target).sum().item()
            sample_num += len(prediction)
            
        loss = total_loss / test_batch_num
        acc = correct / sample_num
        return loss, acc

In [15]:
# 读取数据
data_dir = './data'

# 定义一个transform操作，用户将torch中的数据转换为可以输入到我们模式的形式
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) 
    ]
) # 将数据归一化

#获取cifar-10数据集并进行transform
cifar_train = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True,transform=transform)
cifar_test = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True,transform=transform)

# cifar-10数据集对应10个类别
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

num_classes = 10
epochs = 100
lr = 0.001
batch_size = 2
device = torch.device('cpu') # 如使用0号GPU则填写 ‘cuda:1’

#生成dataloader
cifar_trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers = 0)
cifar_testloader = torch.utils.data.DataLoader(cifar_test, batch_size=512, shuffle=True, num_workers=0)

# 初始化模型
net = MyConvModule().to(device)

# 使用多元交叉熵损失
criterion = nn.CrossEntropyLoss()

# 使用Adam优化器
optimizer = optim.Adam(net.parameters(), lr=lr)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# 训练模型
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

for epoch in range(epochs):
    # 在训练集上训练
    print('before train')
    train_loss, train_acc = train_epoch(net, data_loader = cifar_trainloader, device = device)
    print('after train')
    # 在测试集上测试
    test_loss, test_acc = test_epoch(net, data_loader=cifar_testloader, device = device)
    # 保存各指标
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    test_loss_list.append(test_loss)
    test_acc_list.append(test_acc)
    print(f'epoch: {epoch}\t train_loss: {train_loss:.4f} \t'
          f'train_acc:{train_acc}\t'
          f'test_loss:{test_loss:.4f} \t test_acc:{test_acc}')

before train
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size(

output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.s

output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.s

output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.s

output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.shape torch.Size([2, 10]) target.shape torch.Size([2])
output.s

In [ ]:
#画图
x = np.linspace(0, len(train_loss_list), len(train_loss_list))
plt.plot(x, train_loss_list, label = 'train_loss', linewidth=1.5)
plt.plot(x, test_loss_list, label='test_loss', linewidth = 1.5)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

y = np.linspace(0, len(train_acc_list), len(train_acc_list))
plt.plot(y, train_acc_list, label = 'train_acc', linewidth=1.5)
plt.plot(y, test_acc_list, label='test_acc', linewidth = 1.5)
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend()
plt.show()